In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from langchain_groq import ChatGroq
from langchain_openai import ChatOpenAI
os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")
os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY")

/Users/yasjudanulislam/Documents/Agentic AI Developement/Langraph/venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [3]:
llm=ChatGroq(model="gemma2-9b-it")
llm.invoke("Hello")

AIMessage(content='Hello! 👋\n\nHow can I help you today? 😊\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 10, 'total_tokens': 25, 'completion_time': 0.027272727, 'prompt_time': 0.0011744, 'queue_time': 0.187975214, 'total_time': 0.028447127}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--c4b98305-250c-46ba-bb49-75a1043f5d0f-0', usage_metadata={'input_tokens': 10, 'output_tokens': 15, 'total_tokens': 25})

In [4]:
from langgraph.graph import StateGraph,START,END
from typing import TypedDict, Annotated
from langchain_core.messages import BaseMessage, HumanMessage
from langgraph.graph.message import add_messages


In [5]:
class ChatState(TypedDict):

    messages : Annotated[list[BaseMessage],add_messages]

In [6]:
def chat_node(ChatState):
    messages = ChatState['messages']

    res = llm.invoke(messages)
    return {'messages':[res]}


In [7]:
from langgraph.checkpoint.memory import MemorySaver

## checkpointer to be implemented
checkpointer = MemorySaver()
config = {'configurable':{'thread_id':1}}

graph = StateGraph(ChatState)


# add nodes
graph.add_node('chat_node',chat_node)


graph.add_edge(START,'chat_node')
graph.add_edge('chat_node',END)

In [8]:
chatbot=graph.compile(checkpointer=checkpointer)

In [9]:
initial_state = {
    "messages":"Hey, what is my name?"
}
res = chatbot.invoke(initial_state,config=config)
res['messages'][-1].content

"As an AI, I have no memory of past conversations and do not know your name. If you'd like to tell me, I'm happy to learn! 😊\n"

In [ ]:
while True:

    user_message = input('Type here :')
    print('User :', user_message)

    if user_message.strip().lower() in ['exit','bye','quit']:
        break

    response = chatbot.invoke({'messages':[HumanMessage(content=user_message)]})

    print('AI:',response['message'][-1].content)



In [ ]:

for i in range(0,5):
    user_message = input('Type here :')
    print('User :', user_message)

    if user_message.strip().lower() in ['exit','bye','quit']:

        response = chatbot.invoke({'messages':[HumanMessage(content=user_message)]})

    print('AI:',response['message'][-1].content)

In [ ]:
user_message = input("Typer Here: ")